# Assignment 4.  
## Programming simple SQL queries from a host Programming Language (Python)

In this assigment, we will fetch data from a relational database server directly into Python variables. Using these new skills, we will complete the problems from Chapters 5 and 6 in *SQL Queries for Mere Mortals*.

### Connect to the database 

First, let's assign the login credentials. We will use the python libary `getpass` to enter the password without displaying it.

In [1]:
import getpass

host = "db.data-science-ust.net"
user = "dimitri"    # !! Replace with your user name !!
password = getpass.getpass()

········


Then we will use the **MySQL client library** called `pymysql`. Client libraries provide all the functionality to interact with the database from the client side.

In [2]:
import pymysql

# establish a database connection
conn = pymysql.connect(host=host, user=user, passwd=password)

#### Execute an SQL query

Using the connection object `conn`, we obtain a `cursor`: an object that manages the execution of queries. Cursors can be configured in various ways. This code constructs a cursor that retrieves query output in the form of lists of Python `dict`s.

In [3]:
cursor = conn.cursor(cursor=pymysql.cursors.DictCursor)

Now we can use `cursor` to send queries to the server and retrieve the result. The `execute` method returns the number of items in the query result but the data themselves are still inside `cursor`.

In [4]:
sql = """
SELECT * 
    FROM shared_sales.customers
    ORDER BY CustLastName, CustFirstName
    LIMIT 10
"""
cursor.execute(sql)

10

#### Fetch items (database rows) 
To fetch the next item from the result as `dict`, use the `cursor.fetchone()`. The keys of the dict correspond to column names with row values in the values of the dict.

In [5]:
first_item = cursor.fetchone()
print(first_item)

{'CustomerID': 1014, 'CustFirstName': 'Sam', 'CustLastName': 'Abolrous', 'CustStreetAddress': '611 Alpine Drive', 'CustCity': 'Palm Springs', 'CustState': 'CA', 'CustZipCode': '92263', 'CustAreaCode': 760, 'CustPhoneNumber': '555-2611'}


Call it again to fetch the next item

In [6]:
second_item = cursor.fetchone()
print(second_item)

{'CustomerID': 1020, 'CustFirstName': 'Joyce', 'CustLastName': 'Bonnicksen', 'CustStreetAddress': '2424 Thames Drive', 'CustCity': 'Bellevue', 'CustState': 'WA', 'CustZipCode': '98006', 'CustAreaCode': 425, 'CustPhoneNumber': '555-2726'}


To fetch all the remaining items as a `list` of `dict`s, use `cursor.fetchall()`

In [7]:
items = cursor.fetchall()
items

[{'CustomerID': 1004,
  'CustFirstName': 'Robert',
  'CustLastName': 'Brown',
  'CustStreetAddress': '672 Lamont Ave',
  'CustCity': 'Houston',
  'CustState': 'TX',
  'CustZipCode': '77201',
  'CustAreaCode': 713,
  'CustPhoneNumber': '555-2491'},
 {'CustomerID': 1009,
  'CustFirstName': 'Andrew',
  'CustLastName': 'Cencini',
  'CustStreetAddress': '507 - 20th Ave. E.\nApt. 2A',
  'CustCity': 'Seattle',
  'CustState': 'WA',
  'CustZipCode': '98105',
  'CustAreaCode': 206,
  'CustPhoneNumber': '555-2601'},
 {'CustomerID': 1026,
  'CustFirstName': 'Kirk',
  'CustLastName': 'DeGrasse',
  'CustStreetAddress': '455 West Palm Ave',
  'CustCity': 'San Antonio',
  'CustState': 'TX',
  'CustZipCode': '78284',
  'CustAreaCode': 210,
  'CustPhoneNumber': '555-2311'},
 {'CustomerID': 1019,
  'CustFirstName': 'Zachary',
  'CustLastName': 'Ehrlich',
  'CustStreetAddress': '12330 Kingman Drive',
  'CustCity': 'Glendale',
  'CustState': 'CA',
  'CustZipCode': '91209',
  'CustAreaCode': 818,
  'CustPho

Note that running `execute.fetchall()` the second time yields an empty list: we got the entire result set out of the `cursor`!

In [8]:
cursor.fetchall()

[]

And `cursor.fetchone()` returns `None`.

In [9]:
cursor.fetchone()

You must execute a new query to fill the `cursor` with new contents.

In [10]:
cursor.execute(sql)

10

In addition to the two `fetch` methods, you can use the `cursor` as the generator in a `for` loop to iterate over the elements of the result set:

In [11]:
for item in cursor:
    print("{CustomerID}: {CustLastName}, {CustFirstName}".format(**item))

1014: Abolrous, Sam
1020: Bonnicksen, Joyce
1004: Brown, Robert
1009: Cencini, Andrew
1026: DeGrasse, Kirk
1019: Ehrlich, Zachary
1015: Gehring, Darren
1011: Hallmark, Alaina
1003: Hallmark, Gary
1010: Kennedy, Angel


# Examples

#### Example 1
What is the inventory value of each product (price x quantities)?

In [12]:
cursor.execute("USE shared_sales")  # enter the right database

0

In [13]:
# assign the SQL query string to a variable to use in a query later
sql = """
SELECT ProductNumber, ProductName, QuantityOnHand * RetailPrice as InventoryValue
FROM products
"""

In [14]:
# execute the SQL query
cursor.execute(sql)

40

Now retrieve and show the result:

In [15]:
results = cursor.fetchall()
print(results)

[{'ProductNumber': 1, 'ProductName': 'Trek 9000 Mountain Bike', 'InventoryValue': Decimal('7200.00')}, {'ProductNumber': 2, 'ProductName': 'Eagle FS-3 Mountain Bike', 'InventoryValue': Decimal('14400.00')}, {'ProductNumber': 3, 'ProductName': 'Dog Ear Cyclecomputer', 'InventoryValue': Decimal('1500.00')}, {'ProductNumber': 4, 'ProductName': 'Victoria Pro All Weather Tires', 'InventoryValue': Decimal('1099.00')}, {'ProductNumber': 5, 'ProductName': 'Dog Ear Helmet Mount Mirrors', 'InventoryValue': Decimal('89.40')}, {'ProductNumber': 6, 'ProductName': 'Viscount Mountain Bike', 'InventoryValue': Decimal('3175.00')}, {'ProductNumber': 7, 'ProductName': 'Viscount C-500 Wireless Bike Computer', 'InventoryValue': Decimal('1470.00')}, {'ProductNumber': 8, 'ProductName': 'Kryptonite Advanced 2000 U-Lock', 'InventoryValue': Decimal('1000.00')}, {'ProductNumber': 9, 'ProductName': 'Nikoma Lok-Tight U-Lock', 'InventoryValue': Decimal('396.00')}, {'ProductNumber': 10, 'ProductName': 'Viscount Micr

You may use the `pprint` module from the [standard library](https://docs.python.org/3/library/pprint.html) to pretty-print the resulting dict list:

In [16]:
import pprint
pprint.pprint(results)

[{'InventoryValue': Decimal('7200.00'),
  'ProductName': 'Trek 9000 Mountain Bike',
  'ProductNumber': 1},
 {'InventoryValue': Decimal('14400.00'),
  'ProductName': 'Eagle FS-3 Mountain Bike',
  'ProductNumber': 2},
 {'InventoryValue': Decimal('1500.00'),
  'ProductName': 'Dog Ear Cyclecomputer',
  'ProductNumber': 3},
 {'InventoryValue': Decimal('1099.00'),
  'ProductName': 'Victoria Pro All Weather Tires',
  'ProductNumber': 4},
 {'InventoryValue': Decimal('89.40'),
  'ProductName': 'Dog Ear Helmet Mount Mirrors',
  'ProductNumber': 5},
 {'InventoryValue': Decimal('3175.00'),
  'ProductName': 'Viscount Mountain Bike',
  'ProductNumber': 6},
 {'InventoryValue': Decimal('1470.00'),
  'ProductName': 'Viscount C-500 Wireless Bike Computer',
  'ProductNumber': 7},
 {'InventoryValue': Decimal('1000.00'),
  'ProductName': 'Kryptonite Advanced 2000 U-Lock',
  'ProductNumber': 8},
 {'InventoryValue': Decimal('396.00'),
  'ProductName': 'Nikoma Lok-Tight U-Lock',
  'ProductNumber': 9},
 {'Inve

Or you may like to convert it into a pandas dataframe of display and manipulation.

In [17]:
import pandas as pd
pd.DataFrame.from_records(results, index=["ProductNumber"])

,ProductName,InventoryValue
ProductNumber,,
1,Trek 9000 Mountain Bike,7200.00
2,Eagle FS-3 Mountain Bike,14400.00
3,Dog Ear Cyclecomputer,1500.00
4,Victoria Pro All Weather Tires,1099.00
5,Dog Ear Helmet Mount Mirrors,89.40
6,Viscount Mountain Bike,3175.00
7,Viscount C-500 Wireless Bike Computer,1470.00
8,Kryptonite Advanced 2000 U-Lock,1000.00
9,Nikoma Lok-Tight U-Lock,396.00


This is enough to fulfill the requirement of the assignment but you may use your Python string formatting skills to produce a nicer output or to perform additional operations.

In [18]:
# Execute query again
cursor.execute(sql)

# print heading
item_line = "{ProductNumber:3}: {ProductName:40} {InventoryValue:>8}" 
heading = item_line.format(ProductNumber="#", ProductName="Product Name", InventoryValue="Value")
print(heading)
print('-' * len(heading)) # dividing line

# print items from cursor
total = 0
for item in cursor:
    print(item_line.format(**item))
    total += item['InventoryValue']

# print total    
print('-' * len(heading))
print(item_line.format(ProductNumber='', ProductName='TOTAL:', InventoryValue=total))

#  : Product Name                                Value
------------------------------------------------------
  1: Trek 9000 Mountain Bike                   7200.00
  2: Eagle FS-3 Mountain Bike                 14400.00
  3: Dog Ear Cyclecomputer                     1500.00
  4: Victoria Pro All Weather Tires            1099.00
  5: Dog Ear Helmet Mount Mirrors                89.40
  6: Viscount Mountain Bike                    3175.00
  7: Viscount C-500 Wireless Bike Computer     1470.00
  8: Kryptonite Advanced 2000 U-Lock           1000.00
  9: Nikoma Lok-Tight U-Lock                    396.00
 10: Viscount Microshell Helmet                 720.00
 11: GT RTS-2 Mountain Bike                    8250.00
 12: Shinoman 105 SC Brakes                     376.00
 13: Shinoman Dura-Ace Headset                 1350.00
 14: Eagle SA-120 Clipless Pedals              2799.00
 15: ProFormance Toe-Klips 2G                   199.60
 16: ProFormance ATB All-Terrain Pedal         1120.00
 17: Shino

# Assignment Problems

Note that table names and column names may differ from those in the book. Use `SHOW DATABASES`, `USE <database>`, and `SHOW TABLES` commands to navigate the databases and `SHOW CREATE TABLE <table>` to inspect the declaration of the individual tables.

## In database `shared_sales`.

#### Problem 1 .

Show all products with price reduced by 5%.

#### Problem 2.
Show the list of orders made by each customer in descending order date. Hint: you might need to order by more than one column.

#### Problem 3.
Show vendor name and addresses sorted by vendor name.

## In database `shared_entertain`

#### Problem 4.
Show all customers ordered by city.

#### Problem 5.
List all entertainers and their Web sites.

#### Problem 6.
Show the date of each agent's first six-month performance review.


**Hint:** You will need date arithmetic https://dev.mysql.com/doc/refman/5.7/en/date-and-time-functions.html

## In database `shared_schedule`

#### Problem 7
Show the staff members in descending order of salary.

#### Problem 8
Make the staff phone list.

#### Problem 9
List students ordered by the city they live in.

## In database `shared_bowling`.

#### 10. Show next year's tournament date for each tournament location.

**Hint:** Add 364 days to get the same day of the week. See date and time functions here: https://dev.mysql.com/doc/refman/5.7/en/date-and-time-functions.html

#### 11. List names and phone numbers of the members of the league 
(32 rows)

#### 12. List each team's lineup.
(32 rows)

## In database `shared_sales`

#### 13. List vendors from the cities of Ballard, Bellevue, and Redmond.
(3 rows)

#### 14. Show alphebetized list of products with a retail price of \$1,250.00 or more
(13 rows)

#### 15. List vendors with no websites

## In database `shared_entertain`

#### 16. List engagements that occurred during October 2017
(24 rows)

#### 17. Show engagements in October 2017 that start between noon and 5 pm.

#### 18. Show engagements that start and end on the same day.

## In database `shared_schedule`

#### 19. Show staff members that use a PO Box as their address.

#### 20. Show students who live outside the Pacific Northwest

#### 21. Show subjects with subject code starting with "MUS"

Please review string comparison functions https://dev.mysql.com/doc/refman/5.7/en/string-comparison-functions.html

#### 22. List the ID numbers of all Associate professors who are employed full time.

## In database `shared_bowling`

#### 23. List tournaments held in 2017

#### 24. List tournament schedules for Bolero, Red Rooster, and Thunderbird Lanes.

#### 25. List the bowlers who live in Bellevue, Bothell, Duvall, Redmond, and Woodinville who are on teams 5, 6, 7, or 8.

## In database `shared_recipes`

#### 26. List all recipes for main courses (recipe class 1) that have notes.

#### 27. Display the first five recipes.